In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import os 
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score
import gensim.downloader as api

In [2]:
df = pd.read_csv("outputAllYear.csv")
country_to_continent = {
    "Sweden": "Europe", "South Africa": "Africa", "Armenia": "Asia", "Netherlands": "Europe",
    "Maldives": "Asia", "Macao": "Asia", "Slovenia": "Europe", "Barbados": "North America",
    "Bangladesh": "Asia", "Mexico": "North America", "Taiwan": "Asia", "Georgia": "Asia",
    "Bhutan": "Asia", "South Sudan": "Africa", "Congo": "Africa", "Algeria": "Africa",
    "Belize": "North America", "Zimbabwe": "Africa", "Iran": "Asia", "Hungary": "Europe",
    "Latvia": "Europe", "Belgium": "Europe", "Denmark": "Europe", "Italy": "Europe",
    "Albania": "Europe", "Iceland": "Europe", "Democratic Republic Congo": "Africa",
    "Botswana": "Africa", "Monaco": "Europe", "Mauritius": "Africa", "Guatemala": "North America",
    "Madagascar": "Africa", "Somalia": "Africa", "Togo": "Africa", "Reunion": "Africa",
    "Bulgaria": "Europe", "El Salvador": "North America", "Sri Lanka": "Asia",
    "South Korea": "Asia", "Sudan": "Africa", "Kyrgyzstan": "Asia", "Brunei Darussalam": "Asia",
    "North Macedonia": "Europe", "Bahamas": "North America", "Qatar": "Asia", "Timor-Leste": "Asia",
    "United States": "North America", "Bahrain": "Asia", "Mali": "Africa", "Angola": "Africa",
    "Namibia": "Africa", "India": "Asia", "Venezuela": "South America", "Russian Federation": "Europe",
    "Viet Nam": "Asia", "Canada": "North America", "Cuba": "North America", "Ireland": "Europe",
    "Panama": "North America", "Oman": "Asia", "Senegal": "Africa", "Montenegro": "Europe",
    "Guinea-Bissau": "Africa", "Seychelles": "Africa", "Yemen": "Asia", "Peru": "South America",
    "Pakistan": "Asia", "Chad": "Africa", "Finland": "Europe", "Paraguay": "South America",
    "Yugoslavia": "Europe", "Azerbaijan": "Asia", "Tunisia": "Africa", "Kazakhstan": "Asia",
    "Moldova": "Europe", "Austria": "Europe", "Malaysia": "Asia", "Rwanda": "Africa",
    "Hong Kong": "Asia", "Cameroon": "Africa", "Papua New Guinea": "Oceania",
    "Czech Republic": "Europe", "Cambodia": "Asia", "Gabon": "Africa", "Niger": "Africa",
    "Portugal": "Europe", "Switzerland": "Europe", "New Zealand": "Oceania", "Singapore": "Asia",
    "Trinidad and Tobago": "North America", "Syrian Arab Republic": "Asia", "Jordan": "Asia",
    "France": "Europe", "Cote d'Ivoire": "Africa", "Eritrea": "Africa", "Argentina": "South America",
    "United Arab Emirates": "Asia", "Sierra Leone": "Africa", "Uruguay": "South America",
    "China": "Asia", "Burundi": "Africa", "Cape Verde": "Africa", "Myanmar": "Asia",
    "Egypt": "Africa", "Unknown": "Unknown", "Nicaragua": "North America", "Thailand": "Asia",
    "Liberia": "Africa", "Central African Republic": "Africa", "Puerto Rico": "North America",
    "Ecuador": "South America", "Spain": "Europe", "Brazil": "South America", "Gambia": "Africa",
    "Nepal": "Asia", "Kuwait": "Asia", "Afghanistan": "Asia", "Swaziland": "Africa",
    "Romania": "Europe", "French Guiana": "South America", "Germany": "Europe",
    "Bolivia": "South America", "Australia": "Oceania", "Greece": "Europe", "Uzbekistan": "Asia",
    "Israel": "Asia", "Chile": "South America", "Uganda": "Africa", "Haiti": "North America",
    "Morocco": "Africa", "Poland": "Europe", "Philippines": "Asia", "Mozambique": "Africa",
    "Guyana": "South America", "Zambia": "Africa", "Serbia": "Europe", "Jamaica": "North America",
    "Belarus": "Europe", "Cyprus": "Europe", "United Kingdom": "Europe", "Guinea": "Africa",
    "Honduras": "North America", "Laos": "Asia", "Indonesia": "Asia", "Fiji": "Oceania",
    "Costa Rica": "North America", "Mauritania": "Africa", "Aruba": "North America",
    "Colombia": "South America", "Burkina Faso": "Africa", "Slovakia": "Europe",
    "Dominican Republic": "North America", "Tanzania": "Africa", "Norway": "Europe",
    "Mongolia": "Asia", "Ukraine": "Europe", "Ghana": "Africa", "Iraq": "Asia",
    "Saudi Arabia": "Asia", "Benin": "Africa", "Luxembourg": "Europe",
    "Bosnia and Herzegovina": "Europe", "Lithuania": "Europe", "Ethiopia": "Africa",
    "Libya": "Africa", "Palestine": "Asia", "Lebanon": "Asia", "Croatia": "Europe",
    "Kenya": "Africa", "Malta": "Europe", "Malawi": "Africa", "Estonia": "Europe",
    "Turkey": "Asia", "Japan": "Asia", "Nigeria": "Africa", "Federated States of Micronesia": "Oceania"
}
# ฟังก์ชันเพื่อแปลงประเทศเป็นทวีป
def map_country_to_continent(countries):
    continents = [country_to_continent.get(country, 'Unknown') for country in countries.split(',')]
    return continents

# เพิ่มคอลัมน์ทวีป
df['continent'] = df['item.bibrecord.head.author-group.affiliation.country'].apply(map_country_to_continent)

# คำนวณความถี่ของแต่ละทวีปและแปลงเป็นคอลัมน์ใหม่
def calculate_continent_counts(continents):
    return {continent: continents.count(continent) for continent in set(continents)}

# สร้าง DataFrame จากผลลัพธ์ความถี่
continent_counts = df['continent'].apply(calculate_continent_counts)
continent_df = pd.DataFrame(continent_counts.tolist()).fillna(0).astype(int)

# รวม DataFrame เดิมกับ DataFrame ใหม่
result_df = pd.concat([df, continent_df], axis=1)
result_df.drop(columns='continent', inplace=True)
result_df.drop(columns='item.bibrecord.head.author-group.affiliation.country', inplace=True)

result_df.rename(columns={'coredata.dc:title': 'Title', 'coredata.prism:publicationName': 'PublicationName' , 'item.bibrecord.head.citation-info.citation-language.@language':'Language', 'coredata.citedby-count':'CitedByCount'}, inplace=True)

In [4]:
word2vec = api.load('glove-wiki-gigaword-50')  # ใช้ GloVe vector 50 มิติ

# กรองข้อมูลที่ไม่ว่างเปล่า
result_df = result_df[result_df['authkeywords.author-keyword'].str.strip() != ""]

# แยกคำสำคัญในแต่ละแถว
result_df['keywords_list'] = result_df['authkeywords.author-keyword'].str.split(',')

# สร้าง Word Embeddings สำหรับแต่ละแถว
def get_average_embedding(keywords):
    if not isinstance(keywords, list):
        return np.zeros(word2vec.vector_size)
    
    vectors = [word2vec[word] for word in keywords if word in word2vec]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec.vector_size)

result_df['embedding'] = result_df['keywords_list'].apply(get_average_embedding)

# แปลง embeddings เป็น array
embeddings = np.vstack(result_df['embedding'].values)

# ใช้ K-Means Clustering
n_clusters = 5  # ปรับจำนวนคลัสเตอร์ตามต้องการ
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
result_df['cluster'] = kmeans.fit_predict(embeddings)

# คำนวณ Silhouette Score
silhouette_avg = silhouette_score(embeddings, result_df['cluster'])
print(f"Silhouette Score: {silhouette_avg}")

Silhouette Score: 0.8143762088848177


In [ ]:
# Fill NaN values in 'keywords_list' with an empty list
result_df['keywords_list'] = result_df['keywords_list'].apply(lambda x: x if isinstance(x, list) else [])

for cluster_id in range(n_clusters):
    print(f"Cluster {cluster_id}:")
    cluster_keywords = result_df[result_df['cluster'] == cluster_id]['keywords_list']
    flat_keywords = [keyword for sublist in cluster_keywords for keyword in sublist]
    top_keywords = pd.Series(flat_keywords).value_counts().head(10)
    print(top_keywords)

Cluster 0:
Thailand          51
component         20
COVID-19          19
1                 16
diversity         12
2                 12
reliability       11
survival          10
sustainability    10
aging             10
Name: count, dtype: int64
Cluster 1:
Thailand                                  581
COVID-19                                  210
Hadron-Hadron scattering (experiments)    152
Inflammation                              128
CMS                                        98
Oxidative stress                           81
SARS-CoV-2                                 79
Machine learning                           77
Depression                                 74
Adsorption                                 69
Name: count, dtype: int64
Cluster 2:
inflammation        57
apoptosis           25
cytotoxicity        24
meta-analysis       23
antioxidant         23
biomarkers          23
cytokines           19
oxidative stress    18
neuro-immune        17
Thailand            17
Name: count, dt

In [ ]:
cluster_labels = {
    0: 'Sustainability and Diversity',
    1: 'Medical and Technological Advances',
    2: 'Cell Biology and Biochemistry',
    3: 'Materials Science and Taxonomy',
    4: 'Public Health and Epidemiology',
}

result_df['cluster_label'] = result_df['cluster'].map(cluster_labels)
value_counts = result_df['cluster_label'].value_counts()
result_df.rename(columns={'cluster_label': 'keyword_category'}, inplace=True)
print(value_counts)


cluster_label
Medical and Technological Advances    17034
Materials Science and Taxonomy         1104
Cell Biology and Biochemistry           741
Sustainability and Diversity            676
Public Health and Epidemiology          661
Name: count, dtype: int64


In [ ]:
result_df.drop(columns="authkeywords.author-keyword", inplace=True)
result_df.drop(columns="embedding", inplace=True)
cited_by_count = result_df.pop('CitedByCount')
result_df['CitedByCount'] = cited_by_count

result_df

,Title,PublicationName,Language,Asia,North America,Unknown,South America,Europe,Oceania,Africa,keywords_list,cluster,keyword_category,CitedByCount
0,Public health and international epidemiology f...,"Radiology in Global Health: Strategies, Implem...",English,2,4,0,0,0,0,0,[],1,Medical and Technological Advances,1.0
1,Flexible Printed Active Antenna for Digital Te...,Progress in Electromagnetics Research Symposium,English,0,0,2,0,0,0,0,[],1,Medical and Technological Advances,1.0
2,Parametric study of hydrogen production via so...,Chemical Engineering Science,English,5,0,0,0,0,0,0,"[Circulating fluidized bed, Computational flui...",1,Medical and Technological Advances,21.0
3,Superhydrophobic coating from fluoroalkylsilan...,Applied Surface Science,English,6,0,0,0,0,0,0,"[Encapsulation, Fluoroalkylsilane, Natural rub...",1,Medical and Technological Advances,37.0
4,Electrochemical impedance-based DNA sensor usi...,Analytica Chimica Acta,English,5,1,0,0,0,0,0,"[acpcPNA, Electrochemical impedance spectrosco...",1,Medical and Technological Advances,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211,Long-chain bio-olefins production via oxidativ...,Catalysis Today,English,5,0,0,0,0,0,0,"[Long-chain olefins, Mesoporous KIT-6, Oleic a...",1,Medical and Technological Advances,3.0
20212,Recent Developments and Applications of Microf...,Critical Reviews in Analytical Chemistry,English,0,0,2,0,0,0,0,"[Biological hazards, chemical hazards, food co...",1,Medical and Technological Advances,11.0
20213,"Social justice, education and peacebuilding: c...",Compare,English,1,0,0,0,1,0,0,"[conflict, Education, peacebuilding, social ju...",0,Sustainability and Diversity,5.0
20214,Effects of black soldier fly (Hermetia illucen...,Journal of Applied Aquaculture,English,2,0,0,0,0,0,0,"[Anabas testudineus, Black soldier fly, fish m...",1,Medical and Technological Advances,6.0


In [18]:
result_df.to_csv('result_df.csv', index=False)

End หลังจากนี้โค้ดมายมัง้ ถ้าหาไม่เจอก็อยุใน dump.ipynb


In [17]:
year = 2023
# Path to the folder containing JSON files
folder_path = f'Project/{year}'

# Initialize a list to store the rows of the DataFrame
data_rows = []

# Columns to extract
columns_to_keep = [
    'coredata.srctype',
    'coredata.eid',
    'coredata.dc:description',
    'coredata.pubmed-id',
    'coredata.prism:coverDate',
    'coredata.prism:aggregationType',
    'coredata.prism:url',
    'coredata.source-id',
    'coredata.pii',
    'coredata.citedby-count',
    'coredata.prism:volume',
    'coredata.subtype',
    'coredata.dc:title',
    'coredata.openaccess',
    'coredata.prism:issn',
    'coredata.publishercopyright',
    'coredata.article-number',
    'coredata.subtypeDescription',
    'coredata.prism:publicationName',
    'coredata.prism:doi',
    'coredata.dc:identifier',
    'coredata.dc:publisher',
    'item.bibrecord.head.citation-info.citation-language.@language',
]

# Helper function to safely extract a value from a dictionary
def safe_get(d, keys, default=None):
    try:
        for key in keys:
            d = d.get(key, {})
        return d if d else default
    except AttributeError:
        return default

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.startswith(str(year)) and file_name.endswith(".json"):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            try:
                data = json.load(file)

                # Extract countries from author-group
                author_groups = safe_get(data, ['abstracts-retrieval-response', 'item', 'bibrecord', 'head', 'author-group'], [])
                countries = [
                    safe_get(author, ['affiliation', 'country'], 'Unknown')
                    for author in author_groups
                ]
                countries_string = ','.join(countries)

                # Extract keywords
                auth_keywords = safe_get(data, ['abstracts-retrieval-response', 'authkeywords', 'author-keyword'], [])
                keywords = [keyword.get('$', '') for keyword in auth_keywords if isinstance(keyword, dict)]
                keywords_string = ','.join(keywords) if keywords else 'null'

                # Normalize JSON data and filter columns
                row_data = pd.json_normalize(data.get('abstracts-retrieval-response', {}))
                row = {col: row_data[col].iloc[0] if col in row_data else pd.NA for col in columns_to_keep}

                # Add processed fields
                row['item.bibrecord.head.author-group.affiliation.country'] = countries_string
                row['authkeywords.author-keyword'] = keywords_string

                # Append the row to data_rows
                data_rows.append(row)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")

# Convert the list of rows into a DataFrame
df = pd.DataFrame(data_rows)

# Save the DataFrame to a CSV file
output_path = f'output_{year}.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

df

,coredata.srctype,coredata.eid,coredata.dc:description,coredata.pubmed-id,coredata.prism:coverDate,coredata.prism:aggregationType,coredata.prism:url,coredata.source-id,coredata.pii,coredata.citedby-count,...,coredata.publishercopyright,coredata.article-number,coredata.subtypeDescription,coredata.prism:publicationName,coredata.prism:doi,coredata.dc:identifier,coredata.dc:publisher,item.bibrecord.head.citation-info.citation-language.@language,item.bibrecord.head.author-group.affiliation.country,authkeywords.author-keyword
0,j,2-s2.0-85170238281,Pyrocatechol violet/copper ion-graphene oxide/...,37633552,2023-12-31,Journal,https://api.elsevier.com/content/abstract/scop...,17544,S0141813023032129,0,...,© 2023 Elsevier B.V.,126316,Article,International Journal of Biological Macromolec...,10.1016/j.ijbiomac.2023.126316,SCOPUS_ID:85170238281,Elsevier B.V.,English,"Thailand,Thailand,Thailand","Alzheimer's disease,Hydrogel colorimetric sens..."
1,j,2-s2.0-85169978316,"Herein, unusual and rare coordination behavior...",<NA>,2023-12-15,Journal,https://api.elsevier.com/content/abstract/scop...,24642,S0022286023015065,0,...,© 2023 Elsevier B.V.,136416,Article,Journal of Molecular Structure,10.1016/j.molstruc.2023.136416,SCOPUS_ID:85169978316,Elsevier B.V.,English,"India,India,India,India,Thailand","Copper(II),Hirshfeld calculations,Molecular do..."
2,j,2-s2.0-85165929707,This study assessed the characteristics of a z...,<NA>,2023-12-15,Journal,https://api.elsevier.com/content/abstract/scop...,29419,S0044848623006725,0,...,© 2023 Elsevier B.V.,739898,Article,Aquaculture,10.1016/j.aquaculture.2023.739898,SCOPUS_ID:85165929707,Elsevier B.V.,English,"Thailand,Thailand,Thailand,Thailand","Ammonia removal,Carrying capacity,Microbiome,O..."
3,j,2-s2.0-85167831666,A novel anaerobic baffled biofilm-membrane bio...,37582447,2023-12-10,Journal,https://api.elsevier.com/content/abstract/scop...,25349,S0048969723048738,0,...,© 2023 Elsevier B.V.,166248,Article,Science of the Total Environment,10.1016/j.scitotenv.2023.166248,SCOPUS_ID:85167831666,Elsevier B.V.,English,"Thailand,Thailand","Anaerobic baffled biofilm–MBR (AnBB-MBR),Membr..."
4,j,2-s2.0-85165076456,Safety of aquatic products is one of the impor...,<NA>,2023-12-10,Journal,https://api.elsevier.com/content/abstract/scop...,21100390177,S2352485523002700,0,...,© 2023 Elsevier B.V.,103080,Article,Regional Studies in Marine Science,10.1016/j.rsma.2023.103080,SCOPUS_ID:85165076456,Elsevier B.V.,English,"India,India,India,Israel,Thailand,India,Saudi ...","Contamination,Health risk assessment,Heavy met..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2885,j,2-s2.0-85111945558,Long-chain α-olefins (≥ C10) are normally appl...,<NA>,2023-01-01,Journal,https://api.elsevier.com/content/abstract/scop...,16377,S0920586121003497,3,...,© 2021 Elsevier B.V.,<NA>,Article,Catalysis Today,10.1016/j.cattod.2021.07.034,SCOPUS_ID:85111945558,Elsevier B.V.,English,"Thailand,Thailand,Thailand,Thailand,Thailand","Long-chain olefins,Mesoporous KIT-6,Oleic acid..."
2886,j,2-s2.0-85111408415,"Nowadays, food safety has become a major conce...",34304654,2023-01-01,Journal,https://api.elsevier.com/content/abstract/scop...,23973,<NA>,11,...,"© 2021 Taylor & Francis Group, LLC.",<NA>,Review,Critical Reviews in Analytical Chemistry,10.1080/10408347.2021.1949695,SCOPUS_ID:85111408415,Taylor and Francis Ltd.,English,"Unknown,Unknown","Biological hazards,chemical hazards,food conta..."
2887,j,2-s2.0-85110903700,Education is increasingly becoming central to ...,<NA>,2023-01-01,Journal,https://api.elsevier.com/content/abstract/scop...,12860,<NA>,5,...,© 2021 The Author(s). Published by Informa UK ...,<NA>,Article,Compare,10.1080/03057925.2021.1951666,SCOPUS_ID:85110903700,Routledge,English,"United Kingdom,Thailand","conflict,Education,peacebuilding,social justic..."
2888,j,2-s2.0-85106740832,The effects of replacing fish meal protein wit...,<NA>,2023-01-01,Journal,https://api.elsevier.com/con